In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/test-file/tested.csv")
data

In [ ]:
# When data.isna().sum() is called,
# there are missing samples.
# Too many missing Cabin samples
data = data.drop(columns=["Cabin", "Name"])
for feature in data.keys():
    if data[feature].isna().sum() > 0:
        for idx, sample in enumerate(np.where(data[feature].isna(), True, False)):
            if not sample:
                data = data.replace({feature: np.nan}, data[feature].mean())

In [ ]:
# No more missing samples
data.isna().sum()

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder

transformer = LabelEncoder()

# Predicting feature (y) is Survived
for feature in data.keys():
    if data[feature].dtype not in ["int64", "float64"]:
        data[feature] = transformer.fit_transform(data[feature])
   
X = data.drop(columns=["Survived"])
y = data["Survived"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

param_grid = {"kernel": ("linear", "rbf"), "C": [1, 25, 50]}
grid = GridSearchCV(estimator=SVC(), param_grid=param_grid)
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([("scaler", StandardScaler()), ("svc", SVC(C=1, kernel="linear"))])
pipe.fit(X_train, y_train)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

### Create a correlation matrix to view significant features

In [ ]:
sig_features = data.corr()
sig_features.style.background_gradient(cmap='coolwarm')

In [ ]:
y_pred = pipe.predict(X_test)

for i in range(len(y_test)):
    print(f"Actual: {y_test[i]}; Predicted: {y_pred[i]}")

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
acc